## make adjacency matrix to export

In [104]:
from pathlib import Path
import os
import numpy as np
import networkx as nx

btw, pathlib.Path is amazing, just started playing with it, fully object-oriented path manipulation

### set up paths, etc

In [105]:
p = Path('desikan_res-1x1x1')

In [106]:
# making sure I know what iterdir() does
[i for i in p.iterdir()][:2]

[PosixPath('desikan_res-1x1x1/sub-NDARCA789EE0_acq-64dir_dwi_desikan_res-1x1x1_measure-spatial-ds._elist.csv'),
 PosixPath('desikan_res-1x1x1/sub-NDARAK187ZLP_acq-64dir_dwi_desikan_res-1x1x1_measure-spatial-ds._elist.csv')]

### grab graphs

In [107]:
# list of numpy matrices
graphs = [nx.read_weighted_edgelist(i, delimiter=',') 
          for i in p.iterdir()]

### check nodelists, output sorted version to numpy matrix

In [108]:
graphs[0].nodes()

NodeView(('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '69', '70', '68', '55'))

In [109]:
graphs[1].nodes()

NodeView(('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70'))

check the below list comprehension, because I'm not 100% confident it's right with edgelist sorting

In [110]:
graphs = [nx.to_numpy_array(g, nodelist=sorted(list(graphs[i].nodes())))
          for i, g in enumerate(graphs)]

### remove diagonals

In [111]:
graphs[0]

array([[1.98015e+05, 8.03400e+03, 4.95760e+04, ..., 1.22000e+02,
        1.74620e+04, 2.73930e+04],
       [8.03400e+03, 4.43440e+04, 2.59400e+03, ..., 0.00000e+00,
        2.95020e+04, 6.06700e+03],
       [4.95760e+04, 2.59400e+03, 5.45240e+04, ..., 1.16000e+02,
        4.51800e+03, 1.35650e+04],
       ...,
       [1.22000e+02, 0.00000e+00, 1.16000e+02, ..., 2.89700e+03,
        0.00000e+00, 0.00000e+00],
       [1.74620e+04, 2.95020e+04, 4.51800e+03, ..., 0.00000e+00,
        8.06920e+04, 7.78800e+03],
       [2.73930e+04, 6.06700e+03, 1.35650e+04, ..., 0.00000e+00,
        7.78800e+03, 6.09490e+04]])

In [112]:
for i, _ in enumerate(graphs):
    np.fill_diagonal(graphs[i], 0)

In [113]:
graphs[0]  # cool beans

array([[    0.,  8034., 49576., ...,   122., 17462., 27393.],
       [ 8034.,     0.,  2594., ...,     0., 29502.,  6067.],
       [49576.,  2594.,     0., ...,   116.,  4518., 13565.],
       ...,
       [  122.,     0.,   116., ...,     0.,     0.,     0.],
       [17462., 29502.,  4518., ...,     0.,     0.,  7788.],
       [27393.,  6067., 13565., ...,     0.,  7788.,     0.]])

don't have to worry about nodelists getting removed here, cause networkx fills them in

### consider LLC somehow

graphs is currently a list of numpy arrays, where each element is the per-subject adjacency matrix

In [99]:
# ???

### make single output graph

In [100]:
# for each element of graphs, flatten that element
# then, append that element as a row in the final output matrix

### save as csv

In [ ]:
# np.savetxt('HBN_output.csv', graph, delimiter=",")
# then, open in R and do the thing